In [1]:
import time
from time import perf_counter as timer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics

from tensorflow import keras

from keras.layers import Dense 
from keras.layers import Conv2D 
from keras.layers import MaxPooling2D 
from keras.layers import Flatten , Dropout

from sklearn.model_selection import train_test_split
from keras.regularizers  import l2
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.models import Sequential

from utility import loadData, plot_accuracy

Using TensorFlow backend.


In [2]:
X_train, y_train, X_test, y_test = loadData()

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [3]:
len(X_train) , len(X_test)

(27455, 7172)

In [4]:
unique_values = np.unique(y_train)
NUM_CLASSES = len(unique_values) + 1
print("Unique Values: ", unique_values)
print("Number of classes:" , NUM_CLASSES)

y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

Unique Values:  [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
Number of classes: 25


In [5]:
model_cnn = Sequential()
model_cnn.add(Conv2D(64, kernel_size=(4,4), input_shape=(28, 28 ,1), padding='same' , activation = 'relu'))
model_cnn.add(Dropout(0.4))
model_cnn.add(MaxPooling2D(pool_size = (2, 2)))

model_cnn.add(Conv2D(64, kernel_size = (4, 4), padding='same' , activation = 'relu'))
model_cnn.add(Dropout(0.4))
model_cnn.add(MaxPooling2D(pool_size = (2, 2)))

model_cnn.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model_cnn.add(Dropout(0.4))
model_cnn.add(MaxPooling2D(pool_size = (2, 2)))

model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation = 'relu'))
model_cnn.add(Dense(NUM_CLASSES, activation = 'softmax'))

In [7]:
epochs = [10, 20, 30 , 40 , 50 , 60 , 70] 
epochs_history = []
for index, epoch in enumerate(epochs):
    
    start = timer()
    model_cnn.compile(loss ='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    history_ = model_cnn.fit(X_train, y_train_categorical, epochs= epoch , batch_size = 128,
                             use_multiprocessing = True, validation_data=(X_test,y_test_categorical))
    
    end = timer()
    elapsed_time = time.gmtime(end - start)
                               
    test_loss = history_.history['val_loss'][-1]
    test_acc = history_.history['val_accuracy'][-1]
    train_loss = history_.history['loss'][-1]
    train_acc =history_.history['accuracy'][-1]
    epochs_history.append([epoch, train_loss, train_acc, test_loss, test_acc , "{0}h, {1}m, {2}s".format(elapsed_timetm_hour, elapsed_time.tm_min, elapsed_time.tm_sec)])
    

df = pd.DataFrame(epochs_history)
df.columns = ['epoch',  'loss', 'accuracy', 'val_loss', 'val_accuracy' , 'time']
df.to_csv('output/CNN_epoch_report.csv')
df

Train on 27455 samples, validate on 7172 samples
Epoch 1/30
27455/27455 [==============================] - 129s 5ms/step - loss: 0.0186 - accuracy: 0.9957 - val_loss: 0.0912 - val_accuracy: 0.9780
Epoch 2/30
27455/27455 [==============================] - 128s 5ms/step - loss: 0.0208 - accuracy: 0.9956 - val_loss: 0.1054 - val_accuracy: 0.9791
Epoch 3/30
27455/27455 [==============================] - 127s 5ms/step - loss: 0.0238 - accuracy: 0.9950 - val_loss: 0.1439 - val_accuracy: 0.9690
Epoch 4/30
27455/27455 [==============================] - 128s 5ms/step - loss: 0.0221 - accuracy: 0.9950 - val_loss: 0.1313 - val_accuracy: 0.9757
Epoch 5/30
27455/27455 [==============================] - 123s 4ms/step - loss: 0.0202 - accuracy: 0.9956 - val_loss: 0.0827 - val_accuracy: 0.9872
Epoch 6/30
27455/27455 [==============================] - 123s 4ms/step - loss: 0.0249 - accuracy: 0.9945 - val_loss: 0.0989 - val_accuracy: 0.9731
Epoch 7/30
27455/27455 [==============================] - 131s 

,epoch,loss,accuracy,val_loss,val_accuracy,time
0,30,0.020982,0.995775,0.082438,0.984244,"(1970, 1, 1, 1, 1, 51, 3, 1, 0)"
1,70,0.018140,0.997195,0.138865,0.974763,"(1970, 1, 1, 2, 31, 23, 3, 1, 0)"


In [6]:
batch_sizes = [16, 32, 64 , 128, 256]
batch_history = []
for index, batch_size in enumerate(batch_sizes):
    
    start = timer()
    
    model_cnn.compile(loss ='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    history_ = model_cnn.fit(X_train, y_train_categorical, epochs= 50 , batch_size = batch_size,
                             use_multiprocessing = True, validation_data=(X_test,y_test_categorical))
    
    end = timer()
    elapsed_time = time.gmtime(end - start)    
    test_loss = history_.history['val_loss'][-1]
    test_acc = history_.history['val_accuracy'][-1]
    train_loss = history_.history['loss'][-1]
    train_acc =history_.history['accuracy'][-1]
    
    batch_history.append([batch_size, train_loss, train_acc, test_loss, test_acc, "{0}h, {1}m, {2}s".format(elapsed_timetm_hour, elapsed_time.tm_min, elapsed_time.tm_sec)])
    

df = pd.DataFrame(batch_history)
df.columns = ['batch_size', 'loss', 'accuracy', 'val_loss', 'val_accuracy', 'time']
df.to_csv('output/CNN_batch_report.csv')
df

Train on 27455 samples, validate on 7172 samples
Epoch 1/10
27455/27455 [==============================] - 138s 5ms/step - loss: 0.9459 - accuracy: 0.6966 - val_loss: 0.5270 - val_accuracy: 0.9035
Epoch 2/10
27455/27455 [==============================] - 143s 5ms/step - loss: 0.1955 - accuracy: 0.9324 - val_loss: 0.2912 - val_accuracy: 0.9484
Epoch 3/10
27455/27455 [==============================] - 135s 5ms/step - loss: 0.1446 - accuracy: 0.9517 - val_loss: 0.2118 - val_accuracy: 0.9453
Epoch 4/10
27455/27455 [==============================] - 139s 5ms/step - loss: 0.1214 - accuracy: 0.9608 - val_loss: 0.2176 - val_accuracy: 0.9463
Epoch 5/10
27455/27455 [==============================] - 131s 5ms/step - loss: 0.1087 - accuracy: 0.9662 - val_loss: 0.1645 - val_accuracy: 0.9565
Epoch 6/10
27455/27455 [==============================] - 135s 5ms/step - loss: 0.1086 - accuracy: 0.9673 - val_loss: 0.1766 - val_accuracy: 0.9583
Epoch 7/10
27455/27455 [==============================] - 137s 

,batch_size,loss,accuracy,val_loss,val_accuracy,time
0,16,0.100850,0.971845,0.127143,0.962911,"(1970, 1, 1, 0, 22, 48, 3, 1, 0)"
1,32,0.060038,0.985576,0.089775,0.977273,"(1970, 1, 1, 0, 22, 31, 3, 1, 0)"
2,64,0.039581,0.991003,0.092149,0.977273,"(1970, 1, 1, 0, 21, 44, 3, 1, 0)"
3,128,0.023235,0.994537,0.099149,0.980619,"(1970, 1, 1, 0, 20, 53, 3, 1, 0)"
4,256,0.010592,0.997560,0.112099,0.978388,"(1970, 1, 1, 0, 20, 21, 3, 1, 0)"
